In [7]:
import pandas as pd
import numpy as np
import re
def col(df, colname = "artist_name"):
    return np.array([int(x == colname) for x in df.columns]).argmax()
def search_artist(string = "--"):
    results = spotify.search(q='artist:'+string, type='artist')
    items = results['artists']['items']
    if len(items) > 0:
        artist = items[0]
        df_artist = pd.Series(artist)
        df_artist = df_artist[df_artist.index.isin(['name', 'followers', 'genres', 'id', 'popularity'])]
        df_artist["followers"] = df_artist["followers"]['total']
        return df_artist
def query_artists(df, lists = [], full = False):
    return pd.concat([query_artist(artists, string = name) for name in lists], axis = 0)
def query_artist(df, string = "--", full = False):
    lists = []
    for i, artist in enumerate(df["artist_name"]):
        if(len(re.findall("(.*?)"+string+"(.*?)", "".join(artist))) != 0):
            if(full):
                lists.append(df.iloc[i])
            else:
                lists.append(df.iloc[i, [col(df, "artist_name"), col(df, "genres")]])
    if(full): 
        return pd.DataFrame(lists, columns = df.columns)
    else:
        return pd.DataFrame(lists, columns = ["artist_name", "genres"])
import ast
def perfect_eval(string):
    try:
        return ast.literal_eval(string)
    except:
        return []
def rate_artist(df, dc):
    df_1 = query_artists(df, lists = dc.index).reset_index()
    df_2 = pd.concat([df_1, pd.Series(dc.values, name = "rate")], axis = 1)
    df_3 = df_2.copy()
    for index, row in df_2.iterrows():
        for genre in row['genres']:
            df_3.at[index, genre] = 1
    df_3 = df_3.fillna(0)
    df_user = df_3["rate"]
    df_art = df_3.drop(["index", "artist_name", "genres", "rate"], axis = 1)
    df_profile = df_art.transpose().dot(df_user)
    return df_profile
def select_artist(artists, dfx):
    listid = []
    for index, row in artists.iterrows():
        for genre in row["genres"]:
            if(genre in dfx.index):
                listid.append(index)
    listid = list(set(listid))
    df_select = artists.iloc[listid, [col(artists, "artist_name"), col(artists, "genres")]]
    df_select_2 = df_select.copy()
    for index, row in df_select.iterrows():
        for genre in row['genres']:
            df_select_2.at[index, genre] = 1
    df_select_2 = df_select_2.fillna(0)
    df_select_2 = df_select_2[dfx.index]
    return df_select_2
def recommend_artist(artists, name_rate):
    dc = pd.Series(name_rate)
    dfx = rate_artist(artists, dc)
    df_select = select_artist(artists, dfx)
    df_rec = ((df_select*dfx).sum(axis=1))/(dfx.sum())
    sery_rec = pd.Series(df_rec, name = "rec").sort_values(ascending = False)
    sery_rec_2 = artists.iloc[sery_rec.index, [col(artists, "artist_name")]]
    sery_rec_res = pd.concat([sery_rec, sery_rec_2], axis = 1)
    sery_res = sery_rec_res[~sery_rec_res["artist_name"].isin(dc.index)].head(10)
    sery_res = sery_res.reset_index()
    return sery_res
def pretty_recommend_artist(artists, name_rate):
    df = recommend_artist(artists, name_rate)
    for index, row in df.iterrows():
        print("Number ", index, ": ", row["artist_name"], " matching ", row["rec"]*100, "%.")
artists = pd.read_csv("spotify_dataset_artist(2).csv")
artists.drop(["Unnamed: 0"], axis = 1, inplace = True)
artists["genres"] = [perfect_eval(genre) for genre in artists["genres"]]

In [30]:
query_artist(artists, string = "Cem", full = False)

,artist_name,genres
5274,Cem Adrian,"[turkish pop, turkish rock, turkish singer-son..."
5275,Cem Pilevneli,[turkish alternative]
5276,Cemetery Sun,"[modern alternative rock, modern rock]"
5277,Cemre Emin,[traprun]
28798,Summer Cem,[german hip hop]


In [32]:
pretty_recommend_artist(artists, name_rate)

Number  0 :  Darin  matching  34.375 %.
Number  1 :  Le P  matching  34.375 %.
Number  2 :  Ceylan Ertem  matching  29.166666666666668 %.
Number  3 :  emre aydın  matching  29.166666666666668 %.
Number  4 :  Rafet El Roman  matching  29.166666666666668 %.
Number  5 :  Dean Martin  matching  28.125 %.
Number  6 :  Tony Bennett  matching  28.125 %.
Number  7 :  Bing Crosby  matching  28.125 %.
Number  8 :  Herb Alpert  matching  28.125 %.
Number  9 :  Brenda Lee  matching  28.125 %.


In [31]:
name_rate = {
    "Frank Sinatra": 9,
    "Müslüm Gürses": 7,
    "Michael Jackson": 6,
    "Céline Dion": 8,
    "Cem Adrian": 7
}